# 1. Lecture Overview


- Estimating cost of equity using CAPM (aka expected return or required rate of return on equity)
    - Estimate the market risk premium


- Multifactor models: The Fama-French three factor model


- Estimating the cost of equity using the Fama-French three factor model
    - Estimate the risk premia on the SMB and HML factors


- Application:
    - Estimate the cost of equity of AAPL, using both the CAPM and the Fama-French three factor model 

# 2. Cost of equity using CAPM

Under the CAPM assumptions, we have: 


$$E[R_{i}] = R_{f} +  \beta_i (E[R_{m}] - R_{f}) $$


Therefore, to estimate the cost of equity for any firm $i$, using the CAPM we need:


- The current risk free rate
    - It is best to try to match the horizon of the risk-free rate with the horizon over which the cost of equity applies 
        - Example: if you want to calculate the cost of equity over the next year, use the yield on a 1-year T-bill as the risk-free rate
        - Example: if you want to calculate the cost of equity over the next 10 years, use the yield on a 10-year T-bill as the risk-free rate
        
        
- The firm's market beta
    - Regress past excess returns of the firm on past excess returns on the market
    
    
- An estimate for the market risk premium
    - Take an average of excess returns on the market (use at least 20 years of data)


# 3. The Fama-French three factor model

The Fama-French three factor model adds two more systematic risk factors in addition to the market return from the single-factor model: 

$$R_{i,t} - R_{f,t} = \alpha_i + \beta_{i,m} (R_{m,t} - R_{f,t}) + \beta_{i,smb} R_{smb} + \beta_{i,hml} R_{hml} + \epsilon_{i,t}$$

where:
- $R_{smb}$ is the return on a portfolio that is long on the smallest firms in the economy and short on the largest firms
- $R_{hml}$ is the return on a portfolio that is long on the firms with highest B/M ratios and short on the firms with the lowest B/M ratios
    - B/M = book value of equity divided by market value of equity
    - High B/M firms are called "value" firms
    - Low B/M firms are called "growth" firms


# 4. Cost of equity using the Fama-French three factor model

Under the Fama-French model, we have: 


$$E[R_{i}] = R_{f} +  \beta_{i,m} (E[R_{m}] - R_{f}) + \beta_{i,smb} E[R_{smb}] + \beta_{i,hml} E[R_{hml}] $$


Therefore, to estimate the cost of equity for any firm $i$, using the Fama-French factor we also need

- The firm's market, SMB, and HML betas
    - Regress past excess returns of the firm on past excess returns on the market and the returns on the SMB and HML factors
    
    
- An estimate for the SMB and HML risk premia
    - Take an average of the returns of the SMB and HML factors (use at least 20 years of data)


# 5. Application

Calculate the cost of equity for AAPL using both the CAPM and the Fama-French three factor model

In [1]:
# Import packages
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import matplotlib.pyplot as plt

### 5.1. Calculate risk premia for the market, SMB, and HML factors

In [2]:
# Download the monthly Fama French factors
ff3f = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench','1926-01-01')[0]/100
ff3f

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07,0.0296,-0.0230,-0.0287,0.0022
1926-08,0.0264,-0.0140,0.0419,0.0025
1926-09,0.0036,-0.0132,0.0001,0.0023
1926-10,-0.0324,0.0004,0.0051,0.0032
1926-11,0.0253,-0.0020,-0.0035,0.0031
...,...,...,...,...
2020-10,-0.0210,0.0444,0.0403,0.0001
2020-11,0.1247,0.0548,0.0211,0.0001
2020-12,0.0463,0.0481,-0.0136,0.0001


In [3]:
# Calculate risk premia
risk_premia = ff3f.mean()
risk_premia

Mkt-RF    0.006827
SMB       0.002101
HML       0.003299
RF        0.002701
dtype: float64

### 5.2. Get data on AAPL returns and combine them with the risk factors

In [4]:
# Download adj price data and drop missing values
prices = yf.download('AAPL', '2010-12-01', '2020-12-31', interval='1mo')['Adj Close'].dropna()
prices

[*********************100%***********************]  1 of 1 completed


Date
2010-12-01      9.922909
2011-01-01     10.438492
2011-02-01     10.865795
2011-03-01     10.721204
2011-04-01     10.771044
                 ...    
2020-08-01    128.394058
2020-09-01    115.438042
2020-10-01    108.510361
2020-11-01    118.667633
2020-12-01    132.492020
Name: Adj Close, Length: 121, dtype: float64

In [5]:
# Calculate returns, drop missing values and convert to dataframe
returns = prices.pct_change().dropna().to_frame()
returns

,Adj Close
Date,
2011-01-01,0.051959
2011-02-01,0.040935
2011-03-01,-0.013307
2011-04-01,0.004649
2011-05-01,-0.006569
...,...
2020-08-01,0.214380
2020-09-01,-0.100908
2020-10-01,-0.060012


In [6]:
# Rename column of returns
returns.rename(columns={'Adj Close': 'AAPL'}, inplace = True)
returns

,AAPL
Date,
2011-01-01,0.051959
2011-02-01,0.040935
2011-03-01,-0.013307
2011-04-01,0.004649
2011-05-01,-0.006569
...,...
2020-08-01,0.214380
2020-09-01,-0.100908
2020-10-01,-0.060012


In [7]:
# Convert the index to "period('M')" (monthly)
returns.index = returns.index.to_period('M')
returns

,AAPL
Date,
2011-01,0.051959
2011-02,0.040935
2011-03,-0.013307
2011-04,0.004649
2011-05,-0.006569
...,...
2020-08,0.214380
2020-09,-0.100908
2020-10,-0.060012


In [8]:
# Combine the datasets
full_data = returns.join(ff3f)
full_data

,AAPL,Mkt-RF,SMB,HML,RF
Date,,,,,
2011-01,0.051959,0.0199,-0.0252,0.0082,0.0001
2011-02,0.040935,0.0349,0.0153,0.0129,0.0001
2011-03,-0.013307,0.0046,0.0258,-0.0176,0.0001
2011-04,0.004649,0.0290,-0.0037,-0.0243,0.0000
2011-05,-0.006569,-0.0127,-0.0058,-0.0205,0.0000
...,...,...,...,...,...
2020-08,0.214380,0.0763,-0.0025,-0.0294,0.0001
2020-09,-0.100908,-0.0363,0.0006,-0.0251,0.0001
2020-10,-0.060012,-0.0210,0.0444,0.0403,0.0001


### Cost of equity using the CAPM

In [9]:
# Run the regression to calculate CAPM beta
y = full_data['AAPL'] - full_data['RF']
X = full_data['Mkt-RF']
X = sm.add_constant(X)
X

,const,Mkt-RF
Date,,
2011-01,1.0,0.0199
2011-02,1.0,0.0349
2011-03,1.0,0.0046
2011-04,1.0,0.0290
2011-05,1.0,-0.0127
...,...,...
2020-08,1.0,0.0763
2020-09,1.0,-0.0363
2020-10,1.0,-0.0210


In [10]:
# Run the CAPM regression
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.301
Method:                 Least Squares   F-statistic:                     52.34
Date:                Fri, 02 Apr 2021   Prob (F-statistic):           5.09e-11
Time:                        20:12:12   Log-Likelihood:                 156.83
No. Observations:                 120   AIC:                            -309.7
Df Residuals:                     118   BIC:                            -304.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0122      0.006      1.953      0.0

In [11]:
# Extract AAPL's beta under the CAPM
beta_capm = results.params['Mkt-RF']
beta_capm

1.062056931649567

In [12]:
# Calculate cost of equity (remember this is monthly)
rf = 0.001/12 #current one-year tbill, on a monthly basis

coe_capm = rf + beta_capm * risk_premia['Mkt-RF']
coe_capm

0.007333928692237433

In [13]:
# Annualize the cost of equity
annual_coe_capm = 12 * coe_capm
print(f'\n\n AAPL cost of equity using CAPM = {annual_coe_capm: .4f}')



 AAPL cost of equity using CAPM =  0.0880


### Cost of equity using the Fama-French three factor model

In [14]:
# Run the regression to calculate CAPM beta
y = full_data['AAPL'] - full_data['RF']
X = full_data[['Mkt-RF','SMB','HML']]
X = sm.add_constant(X)
#X

In [15]:
# Run Fama-French three-factor regression
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     24.53
Date:                Fri, 02 Apr 2021   Prob (F-statistic):           2.30e-12
Time:                        20:12:12   Log-Likelihood:                 164.28
No. Observations:                 120   AIC:                            -320.6
Df Residuals:                     116   BIC:                            -309.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0065      0.006      1.066      0.2

In [16]:
# Extract betas unde the three-factor model
beta_mkt = results.params['Mkt-RF']
beta_smb = results.params['SMB']
beta_hml = results.params['HML']
print(beta_mkt, beta_smb, beta_hml)

1.282062406706675 -0.5699329888338278 -0.6737848724904661


In [17]:
# Calculate cost of equity (remember this is monthly)
rf = 0.001/12 #current one-year tbill, on a monthly basis
coe_ff3 = rf + beta_mkt * risk_premia['Mkt-RF'] + beta_smb * risk_premia['SMB'] + beta_hml * risk_premia['HML']
coe_ff3

0.005415889721562294

In [18]:
# Annualize the cost of equity
annual_coe_ff3 = 12 * coe_ff3
print(f'\n\n AAPL cost of equity using Fama-French three factor model = {annual_coe_ff3: .4f}')



 AAPL cost of equity using Fama-French three factor model =  0.0650


**Example**

Recalculate the risk premia of all three risk factors using only data since 1990. 
Now recalculate the cost of equity of AAPL using these new risk-premia estimates.

In [19]:
# Download data on risk factors starting in 1990
ff3f_new = pdr.DataReader('F-F_Research_Data_Factors', 'famafrench', '1990-01-01')[0]/100

# Recalculate risk premia
newrp = ff3f_new.mean()
newrp

Mkt-RF    0.007180
SMB       0.001491
HML       0.000772
RF        0.002164
dtype: float64

In [20]:
# Calculate cost of equity using CAPM
new_coe_capm = (rf + beta_capm * newrp['Mkt-RF']) * 12
new_coe_capm

0.09250955136636616

In [21]:
# Calculate cost of equity using FF 3 factor model
new_coe_ff3f = (rf + beta_mkt * newrp['Mkt-RF'] + beta_smb * newrp['SMB'] + beta_hml * newrp['HML']) * 12
new_coe_ff3f

0.09501984971559671